<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataScience/blob/main/20250124/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習

## 20241224

#本日の講義の目的

機械学習による分類手法に関する説明を行う。
ここでは、代表的な手法である
**決定木**および**ランダムフォレスト**
と呼ばれる2つの分類モデルの構築法を理解する。

なお、決定木やランダムフォレストは2クラス（2値）の分類だけでなく、
3クラス以上の一般のカテゴリ分類や、数値予測にも適応が可能である。

利用するモジュールは以下のようになる。


アルゴリズム(AI) | カテゴリ予測 | 数値予測
-- | -- | --
重回帰 | - | from sklearn.linear_model import LinearRegression
ロジスティック回帰 | from sklearn.linear_model import LogisticRegression | -
決定木  | from sklearn.tree import DecisionTreeClassifier | from sklearn.tree import DecisionTreeRegressor
ランダムフォレスト | from sklearn.ensemble import RandomForestClassifier | from sklearn.ensemble import RandomForestRegressor


### 【参考】

*  決定木ライブラリー

 https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier


* ランダムフォレストライブラリー

 https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


# 準備


## 必要なライブラリーのインポート

In [ ]:
# インストール
!pip install japanize-matplotlib

# 必要なライブラリをインポート
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

# 決定木に必要なライブラリー
from sklearn.tree import DecisionTreeClassifier, plot_tree

# ランダムフォレストに必要なライブラリー
from sklearn.ensemble import RandomForestClassifier

# その他必要なライブラリー
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Cancerデータを使った例

## データ

sklean ライブラリーに格納されているサンプルデータ（cancer data）を使い
**決定木**の説明を行う。

まず、データを読み込む。

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
df['diagnosis'] = cancer.target
df.info()

これは、30項目の細胞に関連したデータ（サイズ、形状、など）と
そこから判断された癌の診断結果（'diagnosis'）のデータである。

全ての項目は数値型で、
特に診断結果（'diagnosis'）は

* 悪性の場合には 0
* 良性の場合には 1

が入力されており、
このような項目を**2値**の項目と呼ぶ。

このデータを学習させて、
細胞に関する30項目を説明変数として
**癌の診断（良性か悪性かの分類）を行うAIを構築**する。

### 基礎集計

In [ ]:
df.describe()

diagnosis は数値型の項目ではあるが、値は 0 と 1 の2値となっていることが確かめられる。


In [ ]:
df['diagnosis'].value_counts()

ヒストグラムを作成して、各項目の分布を確認する。
hue を使いdiagnosisで色分けしてみる

In [ ]:
sns.histplot(data=df, x='mean radius', hue='diagnosis', kde=True)

このグラフからは、mean radius がおよそ18をこえるとほぼ全てが悪性、
逆に10を下回るとほぼ全てが良性になっていることが分かる。

一方で、10から18の間では、良性も悪性もあり、この項目だけでは
悪性か良性かを判断できないことが分かる。

# 決定木
30項目の数値データと癌の診断結果（diagnosis）から、
悪性か良性かを分類するAIを構築する。

悪性か良性かといった分類問題に対する代表的な機械学習の手法として、
**「決定木」**の使い方を説明する。

## 変数

今までの予測と同様に、説明変数と目的変数を定める。

* **説明変数：** 予測する元になる項目。上のデータでは30項目の数値型データがある。
* **目的変数：** 予測の対象となる項目。上のデータでは diagnosis である。

なお、決定木の場合には、**説明変数の標準化は不要である**。


説明変数と目的変数をそれぞれ別の変数（X と y）にセットする。

In [ ]:
# 項目'diagnosis'は変数dfの最後にある項目なので、それを除くと説明変数のみとなる
X = df.iloc[:,:-1]
y = df[['diagnosis']]

In [ ]:
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split

# データを学習用データと検証用データに分割
# test_size は、0.2 から 0.3 程度が良いとされる
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

##決定木モデルを学習する

In [ ]:
# 決定木に必要なライブラリー
from sklearn.tree import DecisionTreeClassifier

# 決定木のインスタンスを作成
model = DecisionTreeClassifier(max_depth=2, random_state=0)

# 学習
model.fit(X_train, y_train.values.ravel())

決定木の内部では、以下の図に示すような
予測がモデルが構築される。

In [ ]:
# 決定木を可視化する
plt.figure(figsize=(16,6))
plot_tree(model, filled=True, feature_names=cancer.feature_names, class_names=['0（悪性）','1（良性）'], fontsize=14)
plt.title("Decision Tree of Cancer Dataset")
plt.show()

決定木モデルでは、説明変数の値の大小で
上の図のように枝分かれしながら予測が行われる。
上の図の場合は
max_depth=2 と指定したので、
2回枝分かれをするモデルになっている。
データが不等式の条件を満たせば左、満たさない場合は右方向に枝を下方向に
たどりながら、最後に到達した箱の class が予測結果である。

上の場合では、枝分かれに用いられた項目は
worst concave points
と
worst area
の2つであった。
この二つが目的変数（この例では良性か悪性か）の判断に有効である、
との結果になったことを意味している。

例えば、ある細胞のデータが、

* worst concave points = 0.2
* worst area = 700

であった場合、予測結果は class=1 (良性)と判定される。





In [ ]:
sns.histplot(data=df, x='worst concave points', hue='diagnosis', kde=True)

In [ ]:
sns.histplot(data=df, x='worst area', hue='diagnosis', kde=True)

In [ ]:
sns.jointplot(x='worst concave points', y='worst area', data=df, hue='diagnosis')

### 評価用のデータを使い予測精度を確認する

In [ ]:
# テストデータで予測
y_pred = model.predict(X_val)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

混同行列（confusion matrix）でも確認する。

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, y_pred)

# Titanic データを使った例（決定木）

In [ ]:
# データを読み込む
data = sns.load_dataset("titanic") #タイタニックのデータ
data.info()
data.head(5)

##データの説明

列名 | 意味
---  | ---
survived	| 生存フラグ（0=死亡、1=生存）
pclass	| チケットクラス（1stクラス、2ndクラス、3rdクラス）
sex	| 性別（male=男性、female＝女性）
sge	| 年齢
sibsp	| タイタニックに同乗している兄弟/配偶者の数
parch	| タイタニックに同乗している親/子供の数
fare	| 料金
embarked	| 出港地（タイタニックへ乗った港）(C=Cherbourg、Q=Queenstown、S=Southampton)
class | 乗船クラス
who |男性 or 女性
adult_male | 成人男性であるかどうか
deck | 乗船していたデッキ
embark_town | 出港地
alive | 生存したかどうか
alone | 一人であったかどうか

In [ ]:
# 欠損値を埋める
data['age'].fillna(data['age'].mean(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data['embark_town'].fillna(data['embark_town'].mode()[0], inplace=True)
# 結果を確認
data.isnull().sum()

In [ ]:
# カテゴリ変数をOne Hot Encoding
data_encoded = pd.get_dummies(data[['sex', 'embarked', 'class', 'who', 'adult_male', 'alone']], drop_first=True)

# 説明変数と目的変数を設定
X = pd.concat([data[['age', 'fare', 'sibsp', 'parch']], data_encoded], axis=1)
y = data['survived']

ハイパーパラメータの max_depth を 2 として決定木を作る。

In [ ]:
# データを訓練セットとテストセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 最適なモデルを作成
model = DecisionTreeClassifier(max_depth=2, random_state=0)
model.fit(X_train, y_train.values.ravel())

# 検証用データで予測
y_pred = model.predict(X_val)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# 決定木を可視化する
plt.figure(figsize=(10,5))
plot_tree(model, filled=True,  fontsize=14, feature_names=X.columns[:-1], class_names=['0(死亡)','1(生存)'])
plt.show()

一番上の who_man は、カテゴリ項目 who を One hot encoding した項目で、
who の値が man であれば 1 、そうでないなら 0 になっている。

who_man <= 0.5 という不等式なので、0 （すなわち man でない）ならば左側に、
1 ならば右側に枝分かれする。

例えば、
* who = woman
* class = Third
の場合は「死亡」と予測されることになる。


### ハイパーパラメータチューニング

決定木のハイパーパラメータは max_depth である。
max_depth を 2から20の範囲で
以下のようにグリッドサーチを行う。


In [ ]:
from sklearn.model_selection import GridSearchCV

# ハイパーパラメータの候補
param_grid = {
    'max_depth': [2, 3, 4, 5, 10, 20]
    }

# グリッドサーチで最適なハイパーパラメータを探す
grid_search = GridSearchCV( DecisionTreeClassifier(), param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train.values.ravel())

最適なパラメータは grid_search.best_estimator_ にセットされる。

In [ ]:
grid_search.best_params_

In [ ]:

# 最適なモデルを作成
model = grid_search.best_estimator_
model.fit(X_train, y_train.values.ravel())

# テストデータで予測
y_pred = model.predict(X_val)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# 決定木を可視化する
plt.figure(figsize=(15,7))
plot_tree(model, filled=True,  fontsize=13, feature_names=X.columns[:-1], class_names=['0(死亡)','1(生存)'])
plt.show()

---
---
---


# ランダムフォレスト
分類問題に対する代表的な機械学習の手法として、
**「ランダムフォレスト」**の使い方を説明する。

ランダムフォレストは、決定木を複雑に組み合わせた構造を持ち、
高い予測精度となるように工夫されてる。
決定木と同様に、カテゴリ型の予測だけでなく
数値型の予測も行える高精度な万能な予測手法である。
ビジネスでも多く用いられている手法である。

ただし、決定木のように予測の内部構造を可視化することができない。
また、ランダムフォレストを利用するためには、
複数のハイパーパラメータの調整が必要で、
他の機械学習法と比べて多くの計算時間を必要とする。


##ランダムフォレストモデルを学習する

学習を行う関数は、今までと同様で fit であり、
以下のように用いる

```
model = RandomForestClassifier()
model.fit(説明変数, 目的変数)
```

である。

Titanic Data を使い学習を行い検証用のデータで精度を評価する。

In [ ]:
# ランダムフォレストに必要なライブラリー
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレストのインスタンスを作成
model = RandomForestClassifier()

# 学習
model.fit(X_train, y_train.values.ravel())

In [ ]:
from sklearn.metrics import accuracy_score

# 評価データで予測
y_pred = model.predict(X_val)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
accuracy

ランダムフォレストでは、説明変数の重要度を表示できる。
値が大きいほど、予測精度に寄与する度合いが大きいことを意味している。
重要度の小さな変数を用いなくてもほとんど精度に影響がない、
逆に重要度の大きな変数を用いない場合は精度が悪化する。

重要度の値は全て正となり、回帰係数とは異なる意味合いの物である。

In [ ]:
model.feature_importances_

In [ ]:
# 特徴量の重要度を可視化
sns.barplot(x=model.feature_importances_, y=X.columns)
plt.show()

## ハイパーパラメータチューニング

ランダムフォレストを利用するためには、
いくつかのハイパーパラメータを適切に設定する必要がある。
上の例ではデフォルト値が用いられているが、
変更することでより精度を向上させることが可能である。

調整することで精度の向上が期待できるパラメーターには以下のものがある。

*  n_estimators（木の数）: 10 ～ 1000 程度
*  max_depth（木の深さ） : 3 ～ 100 程度
*  min_samples_split（分割の最小のサンプル数）: 2 ～ 100 程度

In [ ]:
# ランダムフォレストに必要なライブラリー
from sklearn.ensemble import RandomForestClassifier

# ハイパーパラメータの候補
param_grid = {
    'n_estimators': [20, 50, 100, 200],
    'max_depth': [3, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10]
}

# グリッドサーチで最適なハイパーパラメータを探す
grid_search = GridSearchCV( RandomForestClassifier(), param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train.values.ravel())

# 最適なハイパーパラメータを出力
print("Best Parameters: ", grid_search.best_params_)

# 最適なモデルを作成
model = grid_search.best_estimator_

# 学習
model.fit(X_train, y_train.values.ravel())


## 精度の評価

評価用のデータセットを用いて精度の評価を行う。


In [ ]:
from sklearn.metrics import accuracy_score

# 評価データで予測
y_pred = model.predict(X_val)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
accuracy

検証用データの場合で、混同行列を作成する。

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, y_pred)

---
---
---


# Iris データを使った例

## データ

アイリスデータセットには、
3種類のアイリス花（セトサ、バーシクル、バージニカ）各50レコード
の計150レコードが記録されている。

各項目は以下の通り

項目名 | 型 | 説明
--  | -- | --
Petal Length | 数値 | 花びらの長さ
Petal Width  | 数値 | 花びらの幅
Sepal Length | 数値 | がく片の長さ
Sepal Width | 数値 | がく片の幅
species     | カテゴリ | 花の種類（セトサ、バーシクル、バージニカ）

花びらの長さなど4つの項目から花の種類を予測するAIを作る。

In [ ]:
# seabornからデータをロードする
df = sns.load_dataset('iris')
df.info()
df.head()

## 決定木を使った花の種類の予測

In [ ]:
# 特徴量とラベルを分ける
X = df.drop('species', axis=1)
y = df['species']

# データを学習セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 決定木のインスタンスを作成
model = DecisionTreeClassifier(max_depth=2)

# 学習
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 精度を計算
print("Accuracy: ", accuracy_score(y_test, y_pred))


## 決定木を可視化する

決定木モデルの枝分かれの様子を可視化する

In [ ]:
# 決定木を可視化する
plt.figure(figsize=(10,5))
plot_tree(model, filled=True,  fontsize=12, feature_names=df.columns[:-1], class_names=df['species'].unique() )
plt.show()

In [ ]:
# 特徴量とラベルを分ける
X = df.drop('species', axis=1)
y = df['species']

# データを学習セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 決定木のインスタンスを作成
model = DecisionTreeClassifier(max_depth=2)

# 学習
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 精度を計算
print("Accuracy: ", accuracy_score(y_test, y_pred))


# ハイパーパラメータの候補
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# グリッドサーチで最適なハイパーパラメータを探す
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

# 最適なハイパーパラメータを出力
print("Best Parameters: ", grid_search.best_params_)

# 最適なモデルを取得
best_rf = grid_search.best_estimator_

# 学習
best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

# 特徴量の重要度を可視化
feature_importances = best_rf.feature_importances_
sns.barplot(x=feature_importances, y=X.columns)
plt.title('Feature Importances')
plt.show()




---



---

